In [0]:
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.appName("agg").getOrCreate()

In [0]:
df = spark.read.csv('/FileStore/tables/SalesRecords.csv',inferSchema=True,header=True)

In [0]:
df.printSchema()
df.show(5)

root
-- Region: string (nullable = true)
-- Country: string (nullable = true)
-- Item Type: string (nullable = true)
-- Sales Channel: string (nullable = true)
-- Order Priority: string (nullable = true)
-- Order Date: string (nullable = true)
-- Order ID: integer (nullable = true)
-- Ship Date: string (nullable = true)
-- Units Sold: integer (nullable = true)
-- Unit Price: double (nullable = true)
-- Unit Cost: double (nullable = true)
-- Total Revenue: double (nullable = true)
-- Total Cost: double (nullable = true)
-- Total Profit: double (nullable = true)

+--------------------+--------------------+---------------+-------------+--------------+----------+---------+---------+----------+----------+---------+-------------+----------+------------+
 Region| Country| Item Type|Sales Channel|Order Priority|Order Date| Order ID|Ship Date|Units Sold|Unit Price|Unit Cost|Total Revenue|Total Cost|Total Profit|
+--------------------+--------------------+---------------+-------------+--------------+----------+---------+---------+----------+----------+---------+-------------+----------+------------+
Australia and Oce...| Tuvalu| Baby Food| Offline| H| 5/28/2010|669165933|6/27/2010| 9925| 255.28| 159.42| 2533654.0| 1582243.5| 951410.5|
Central America a...| Grenada| Cereal| Online| C| 8/22/2012|963881480|9/15/2012| 2804| 205.7| 117.11| 576782.8| 328376.44| 248406.36|
 Europe| Russia|Office Supplies| Offline| L| 5/2/2014|341417157| 5/8/2014| 1779| 651.21| 524.96| 1158502.59| 933903.84| 224598.75|
 Sub-Saharan Africa|Sao Tome and Prin...| Fruits| Online| C| 6/20/2014|514321792| 7/5/2014| 8102| 9.33| 6.92| 75591.66| 56065.84| 19525.82|
 Sub-Saharan Africa| Rwanda|Office Supplies| Offline| L| 2/1/2013|115456712| 2/6/2013| 5062| 651.21| 524.96| 3296425.02|2657347.52| 639077.5|
+--------------------+--------------------+---------------+-------------+--------------+----------+---------+---------+----------+----------+---------+-------------+----------+------------+
only showing top 5 rows

In [0]:
df =df.withColumnRenamed('Sales Channel','SalesChannel')

In [0]:
df.columns

Out[11]: ['Region',
 'Country',
 'Item Type',
 'SalesChannel',
 'Order Priority',
 'Order Date',
 'Order ID',
 'Ship Date',
 'Units Sold',
 'Unit Price',
 'Unit Cost',
 'Total Revenue',
 'Total Cost',
 'Total Profit']

In [0]:
#grouping data
df.groupBy('SalesChannel')

Out[12]: <pyspark.sql.group.GroupedData at 0x7f793115a5d0>

In [0]:
#finding min max using group by
df.groupBy('Country').max('Units Sold').show()

+----------+---------------+
 Country|max(Units Sold)|
+----------+---------------+
 Russia| 1779|
 Senegal| 6593|
 Kiribati| 5398|
 Djibouti| 8656|
 Malaysia| 6267|
 Fiji| 9905|
 Comoros| 962|
 Rwanda| 5062|
 France| 1815|
 Sri Lanka| 6952|
 Slovakia| 171|
 Angola| 4187|
 Moldova | 5070|
San Marino| 4750|
East Timor| 5908|
 Lesotho| 9606|
 Albania| 2269|
Madagascar| 7342|
 Myanmar| 8250|
 Nicaragua| 8156|
+----------+---------------+
only showing top 20 rows

In [0]:
df.agg({'Units Sold':'count'}).show()
df.agg({'Units Sold':'max'}).show()
df.agg({'Units Sold':'sum'}).show()

+-----------------+
count(Units Sold)|
+-----------------+
 100|
+-----------------+

+---------------+
max(Units Sold)|
+---------------+
 9925|
+---------------+

+---------------+
sum(Units Sold)|
+---------------+
 512871|
+---------------+

In [0]:
#importing fuctions from spark
from pyspark.sql.functions import countDistinct,avg,stddev

In [0]:
df.select(countDistinct('Country')).show()
df.select(countDistinct('Country').alias('Dist')).show()

+-----------------------+
count(DISTINCT Country)|
+-----------------------+
 76|
+-----------------------+

+----+
Dist|
+----+
 76|
+----+

In [0]:
df.select(stddev('Units Sold')).show()

+-----------------------+
stddev_samp(Units Sold)|
+-----------------------+
 2794.48456169569|
+-----------------------+

In [0]:
#formatting numbers
from pyspark.sql.functions import format_number

In [0]:
unitSold = df.select(stddev('Units Sold').alias('UnitsSold'))

In [0]:
unitSold.select(format_number('UnitsSold',2).alias('round')).show()

+--------+
 round|
+--------+
2,794.48|
+--------+

In [0]:
df.show(5)

+--------------------+--------------------+---------------+------------+--------------+----------+---------+---------+----------+----------+---------+-------------+----------+------------+
 Region| Country| Item Type|SalesChannel|Order Priority|Order Date| Order ID|Ship Date|Units Sold|Unit Price|Unit Cost|Total Revenue|Total Cost|Total Profit|
+--------------------+--------------------+---------------+------------+--------------+----------+---------+---------+----------+----------+---------+-------------+----------+------------+
Australia and Oce...| Tuvalu| Baby Food| Offline| H| 5/28/2010|669165933|6/27/2010| 9925| 255.28| 159.42| 2533654.0| 1582243.5| 951410.5|
Central America a...| Grenada| Cereal| Online| C| 8/22/2012|963881480|9/15/2012| 2804| 205.7| 117.11| 576782.8| 328376.44| 248406.36|
 Europe| Russia|Office Supplies| Offline| L| 5/2/2014|341417157| 5/8/2014| 1779| 651.21| 524.96| 1158502.59| 933903.84| 224598.75|
 Sub-Saharan Africa|Sao Tome and Prin...| Fruits| Online| C| 6/20/2014|514321792| 7/5/2014| 8102| 9.33| 6.92| 75591.66| 56065.84| 19525.82|
 Sub-Saharan Africa| Rwanda|Office Supplies| Offline| L| 2/1/2013|115456712| 2/6/2013| 5062| 651.21| 524.96| 3296425.02|2657347.52| 639077.5|
+--------------------+--------------------+---------------+------------+--------------+----------+---------+---------+----------+----------+---------+-------------+----------+------------+
only showing top 5 rows

In [0]:
#ordering dataset in ascending manner
df.orderBy('Country').show()

+--------------------+-------------+---------------+------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+
 Region| Country| Item Type|SalesChannel|Order Priority|Order Date| Order ID| Ship Date|Units Sold|Unit Price|Unit Cost|Total Revenue|Total Cost|Total Profit|
+--------------------+-------------+---------------+------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+
 Europe| Albania| Clothes| Online| C| 2/2/2010|385383069| 3/18/2010| 2269| 109.28| 35.84| 247956.32| 81320.96| 166635.36|
 Sub-Saharan Africa| Angola| Household| Offline| M| 4/23/2011|135425221| 4/27/2011| 4187| 668.27| 502.54| 2798046.49|2104134.98| 693911.51|
Australia and Oce...| Australia|Office Supplies| Online| C|10/27/2015|158535134|11/25/2015| 2924| 651.21| 524.96| 1904138.04|1534983.04| 369155.0|
Australia and Oce...| Australia| Cereal| Offline| H| 6/9/2013|450563752| 7/2/2013| 682| 205.7| 117.11| 140287.4| 79869.02| 60418.38|
Australia and Oce...| Australia| Beverages| Offline| H| 7/7/2014|240470397| 7/11/2014| 9389| 47.45| 31.79| 445508.05| 298476.31| 147031.74|
 Europe| Austria| Cosmetics| Offline| H| 2/23/2015|868214595| 3/2/2015| 2847| 437.2| 263.33| 1244708.4| 749700.51| 495007.89|
Middle East and N...| Azerbaijan| Cosmetics| Online| M| 2/6/2010|382392299| 2/25/2010| 7234| 437.2| 263.33| 3162704.8|1904929.22| 1257775.58|
Middle East and N...| Azerbaijan|Office Supplies| Online| M| 6/13/2012|423331391| 7/24/2012| 2021| 651.21| 524.96| 1316095.41|1060944.16| 255151.25|
 Asia| Bangladesh| Clothes| Online| L| 1/13/2017|187310731| 3/1/2017| 8263| 109.28| 35.84| 902980.64| 296145.92| 606834.72|
Central America a...| Belize| Clothes| Offline| M| 7/25/2016|807025039| 9/7/2016| 5498| 109.28| 35.84| 600821.44| 197048.32| 403773.12|
 Asia| Brunei|Office Supplies| Online| L| 4/1/2012|320009267| 5/8/2012| 6708| 651.21| 524.96| 4368316.68|3521431.68| 846885.0|
 Europe| Bulgaria| Clothes| Online| M| 4/23/2012|972292029| 6/3/2012| 1673| 109.28| 35.84| 182825.44| 59960.32| 122865.12|
 Europe| Bulgaria|Office Supplies| Online| M| 2/16/2012|189965903| 2/28/2012| 3987| 651.21| 524.96| 2596374.27|2093015.52| 503358.75|
 Sub-Saharan Africa| Burkina Faso| Vegetables| Online| H| 7/17/2012|871543967| 7/27/2012| 8082| 154.06| 90.93| 1245112.92| 734896.26| 510216.66|
 Sub-Saharan Africa| Cameroon| Beverages| Offline| C| 4/1/2015|519820964| 4/18/2015| 5430| 47.45| 31.79| 257653.5| 172619.7| 85033.8|
 Sub-Saharan Africa| Cameroon|Office Supplies| Online| M| 11/7/2011|177636754|11/15/2011| 5518| 651.21| 524.96| 3593376.78|2896729.28| 696647.5|
 Sub-Saharan Africa| Cape Verde| Clothes| Offline| H| 8/2/2014|939825713| 8/19/2014| 4168| 109.28| 35.84| 455479.04| 149381.12| 306097.92|
 Sub-Saharan Africa| Comoros| Cereal| Offline| H| 3/29/2016|902102267| 4/29/2016| 962| 205.7| 117.11| 197883.4| 112659.82| 85223.58|
Central America a...| Costa Rica| Personal Care| Offline| L| 5/8/2017|456767165| 5/21/2017| 6409| 81.73| 56.67| 523807.57| 363198.03| 160609.54|
 Sub-Saharan Africa|Cote d'Ivoire| Clothes| Online| C| 6/8/2012|114606559| 6/27/2012| 3482| 109.28| 35.84| 380512.96| 124794.88| 255718.08|
+--------------------+-------------+---------------+------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+
only showing top 20 rows

In [0]:
#ordering dataset in descending manner
df.orderBy(df['Country'].desc()).show()

+--------------------+---------------+---------------+------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+
 Region| Country| Item Type|SalesChannel|Order Priority|Order Date| Order ID| Ship Date|Units Sold|Unit Price|Unit Cost|Total Revenue|Total Cost|Total Profit|
+--------------------+---------------+---------------+------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+
 Sub-Saharan Africa| Zambia| Snacks| Online| L| 1/4/2011|122583663| 1/5/2011| 4085| 152.58| 97.44| 623289.3| 398042.4| 225246.9|
 Europe| United Kingdom| Household| Online| L| 1/5/2012|955357205| 2/14/2012| 282| 668.27| 502.54| 188452.14| 141716.28| 46735.86|
Australia and Oce...| Tuvalu| Baby Food| Offline| H| 5/28/2010|669165933| 6/27/2010| 9925| 255.28| 159.42| 2533654.0| 1582243.5| 951410.5|
 Asia| Turkmenistan| Household| Offline| L|12/30/2010|441619336| 1/20/2011| 3830| 668.27| 502.54| 2559474.1| 1924728.2| 634745.9|
 Asia| Turkmenistan|Office Supplies| Online| M| 4/23/2013|462405812| 5/20/2013| 5010| 651.21| 524.96| 3262562.1| 2630049.6| 632512.5|
 Sub-Saharan Africa| The Gambia| Meat| Online| M| 1/14/2017|825304400| 1/23/2017| 4767| 421.89| 364.69| 2011149.63|1738477.23| 272672.4|
 Sub-Saharan Africa| The Gambia| Household| Offline| L| 5/26/2012|886494815| 6/9/2012| 2370| 668.27| 502.54| 1583799.9| 1191019.8| 392780.1|
 Sub-Saharan Africa| The Gambia| Baby Food| Offline| M| 2/3/2014|494747245| 3/20/2014| 5559| 255.28| 159.42| 1419101.52| 886215.78| 532885.74|
 Sub-Saharan Africa| The Gambia| Cereal| Offline| H| 6/7/2012|994022214| 6/8/2012| 2117| 205.7| 117.11| 435466.9| 247921.87| 187545.03|
Middle East and N...| Syria| Fruits| Online| L|11/22/2011|162052476| 12/3/2011| 3784| 9.33| 6.92| 35304.72| 26185.28| 9119.44|
 Europe| Switzerland| Cosmetics| Offline| M| 9/17/2012|249693334|10/20/2012| 8661| 437.2| 263.33| 3786589.2|2280701.13| 1505888.07|
 Europe| Switzerland| Personal Care| Online| M|12/23/2010|617667090| 1/31/2011| 273| 81.73| 56.67| 22312.29| 15470.91| 6841.38|
 Asia| Sri Lanka| Cosmetics| Offline| M|11/19/2016|419123971|12/18/2016| 6952| 437.2| 263.33| 3039414.4|1830670.16| 1208744.24|
 Europe| Spain| Household| Offline| L|10/21/2012|213487374|11/30/2012| 4513| 668.27| 502.54| 3015902.51|2267963.02| 747939.49|
 Sub-Saharan Africa| South Sudan| Personal Care| Offline| C|12/29/2013|406502997| 1/28/2014| 2125| 81.73| 56.67| 173676.25| 120423.75| 53252.5|
Australia and Oce...|Solomon Islands| Baby Food| Online| C| 2/4/2015|547995746| 2/21/2015| 2974| 255.28| 159.42| 759202.72| 474115.08| 285087.64|
 Europe| Slovenia| Beverages| Offline| C|10/23/2016|345718562|11/25/2016| 4660| 47.45| 31.79| 221117.0| 148141.4| 72975.6|
 Europe| Slovakia| Vegetables| Online| H| 10/6/2012|759224212|11/10/2012| 171| 154.06| 90.93| 26344.26| 15549.03| 10795.23|
 Sub-Saharan Africa| Sierra Leone|Office Supplies| Offline| M|11/26/2011|441888415| 1/7/2012| 3457| 651.21| 524.96| 2251232.97|1814786.72| 436446.25|
 Sub-Saharan Africa| Sierra Leone|Office Supplies| Offline| H| 12/6/2016|621386563|12/14/2016| 948| 651.21| 524.96| 617347.08| 497662.08| 119685.0|
+--------------------+---------------+---------------+------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+
only showing top 20 rows